In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r'C:\Users\JD\Desktop\Python\fair.csv')

In [3]:
df.isna().sum().sum()

0

In [4]:
df.size

57294

In [5]:
df.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666


In [6]:
#religious:  1>4 crescente
#educ:       9>20 crescente
#occupation: nominale
#affairs:    tempo dedicato relazione extraconiugale

In [7]:
#creo nuova colonna: affairs true/false

df['affairT']=df['affairs']>0

In [8]:
df.affairT.value_counts()

False    4313
True     2053
Name: affairT, dtype: int64

In [9]:
#elimino affairs (affairT creata da qui)

df.drop('affairs',axis=1,inplace=True)

In [10]:
#target

y=df.iloc[:,len(df.columns)-1].values

In [11]:
#correlazione di affairT

df.corr().loc['affairT'].sort_values(ascending=False)

affairT            1.000000
yrs_married        0.203109
children           0.159833
age                0.146519
occupation         0.028981
occupation_husb    0.017637
educ              -0.075280
religious         -0.129299
rate_marriage     -0.331776
Name: affairT, dtype: float64

In [12]:
#ft

x=df.iloc[:,0:8].values

In [13]:
from sklearn.model_selection import train_test_split as split

In [14]:
#divido in train/test

xtr,xts,ytr,yts=split(x,y,test_size=0.2,random_state=1,stratify=y)

In [15]:
from sklearn.model_selection import cross_val_score as cvs

In [16]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier()
import numpy as np

In [17]:
np.mean(cvs(forest,xtr,ytr)) #accuratezza albero decisionale

0.6822492485602627

In [18]:
np.std(cvs(forest,xtr,ytr))  #deviazione standard albero decisionale

0.008745389123696717

In [19]:
#colonne rilevanti secondo albero decisionale

ft=list(df.iloc[:,0:8].columns)
forest.fit(xtr,ytr)
pd.DataFrame({'ft':ft,'imp':forest.feature_importances_}).sort_values('imp',ascending=False)

,ft,imp
7,occupation_husb,0.159843
0,rate_marriage,0.153491
2,yrs_married,0.131410
3,children,0.125774
5,educ,0.117800
6,occupation,0.117597
1,age,0.098209
4,religious,0.095875


## get_dummies

In [20]:
#occupation_husb è una variabile nominale: creo nuove colonne per ogni opzione di occupation_husb
#varrebbe anche per occupation, ma è un dato meno significativo

In [21]:
pd.get_dummies(df.occupation_husb,prefix='occ_h')

,occ_h_1.0,occ_h_2.0,occ_h_3.0,occ_h_4.0,occ_h_5.0,occ_h_6.0
0,0,0,0,0,1,0
1,0,0,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,1,0,0
...,...,...,...,...,...,...
6361,0,0,1,0,0,0
6362,0,0,0,0,1,0
6363,1,0,0,0,0,0
6364,0,0,0,1,0,0


In [22]:
#nuove colonne nel dataset

df=pd.concat((df,pd.get_dummies(df.occupation_husb,prefix='occ_h')),axis=1)

In [23]:
#elimino colonna occupation_husb

df.drop('occupation_husb',axis=1,inplace=True)

In [24]:
#target

y=df.iloc[:,7].values

In [25]:
#ft

ft=list(df.columns)
del ft[7]
x=df[ft].values

In [26]:
xtr,xts,ytr,yts=split(x,y,test_size=0.2,random_state=1,stratify=y)

In [27]:
#nuove colonne rilevanti

forest.fit(xtr,ytr)
pd.DataFrame({'ft':ft,'imp':forest.feature_importances_}).sort_values('imp',ascending=False)

,ft,imp
0,rate_marriage,0.157524
2,yrs_married,0.137955
3,children,0.134609
5,educ,0.131628
6,occupation,0.115328
4,religious,0.108902
1,age,0.103108
10,occ_h_4.0,0.025567
11,occ_h_5.0,0.024897
8,occ_h_2.0,0.022756
